In [ ]:
from drl.environments.VRPTW_Environment import VRPTW_Environment
from stable_baselines3.common.env_util import make_vec_env
wrappable_env = VRPTW_Environment("ortec", "./", seed=7)
N_ENVS = 3
vec_env = make_vec_env(
    lambda: wrappable_env,
    n_envs=N_ENVS,
    vec_env_cls=VRPTW_Environment,
    vec_env_kwargs=dict(start_method="fork"),
)




In [3]:
import pandas as pd
sota_res = pd.read_csv("sota_res.csv")
sota_res_dict = {row["problem"]: (row["distance"], row["vehicle"]) for _, row in sota_res.iterrows()}
result_to_compare = [("ges_1200", "amlt/vrptw_ges_1200/vrptw_ortec/ges_res_ortec_20220708_0405.csv"),
                     ("hybrid", "amlt/vrptw_hybrid_ges_only/vrptw_ortec/hybrid_res_ortec_20220708_0407.csv")]

df = None
for name, file_name in result_to_compare:
    _df = pd.read_csv(file_name)
    _df = _df[["problem", "vehicles", 'total_cost']]
    _df = _df[_df["vehicles"] > 0]
    _df.columns = ["problem", "%s_vehicles"%name, "%s_cost"%name]
    if df is not None: df = pd.merge(df, _df)
    else: df = _df.copy(deep=True)
    
df.loc[:, "sota_vehicles"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (1, 1))[1])
df.loc[:, "sota_cost"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (1, 1))[0])
print(df.shape)
df.mean(axis=0)

FileNotFoundError: [Errno 2] File b'amlt/vrptw_hybrid_120/vrptw_ortec/hybrid_res_ortec_20220706_1102.csv' does not exist: b'amlt/vrptw_hybrid_120/vrptw_ortec/hybrid_res_ortec_20220706_1102.csv'

In [2]:
import os
import pandas as pd

from IPython.display import display


experiment_names = [("GES", "vrptw_ges_1200"), ("HYBRID", "vrptw_hybrid_ges_only"), ("HRBRID_SINGLE", "vrptw_hybrid_single")]
instance_names = ["200", "400", "600", "800", "1000", 'ortec']

sota_res = pd.read_csv("sota_res.csv")
sota_res_dict = {row["problem"]: (row["distance"], row["vehicle"]) for _, row in sota_res.iterrows()}

res_df_list = []

res_stat_list = []
cols = ["instance"]
for experiment, _ in experiment_names:
    cols.extend([f"{experiment}_cost", f"{experiment}_vehicles", f"{experiment}_gap"])
cols.extend(["SOTA_cost", "SOTA_vehicles"])
for instance in instance_names:
    df = None
    for experiment, folder_name in experiment_names:
        for _, _, all_files in os.walk(f"amlt/{folder_name}/vrptw_{instance}/"):
            file_name = sorted(all_files)[-1]
        _df = pd.read_csv(f"amlt/{folder_name}/vrptw_{instance}/{file_name}")
        _df.loc[:, "instance"] = instance
        _df = _df[["problem", "vehicles", 'total_cost', 'instance']]
        _df = _df[_df["vehicles"] > 0]
        _df.columns = ["problem", "%s_vehicles"%experiment, "%s_cost"%experiment, 'instance']
        if df is not None: df = pd.merge(df, _df)
        else: df = _df.copy(deep=True)
    df.loc[:, "sota_vehicles"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (None, None))[1])
    df.loc[:, "sota_cost"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (None, None))[0])
    df.loc[:, "sota_vehicles"].fillna(df[f"{experiment_names[0][0]}_vehicles"], inplace=True)
    df.loc[:, "sota_cost"].fillna(df[f"{experiment_names[0][0]}_cost"], inplace=True)
    for experiment, _ in experiment_names:
        df.loc[:, f'{experiment}_gap'] = df.apply(lambda x: round((x[f'{experiment}_cost']-x['sota_cost'])/x['sota_cost'], 3), axis=1)
    # print(f"************{instance}**********")
    df_mean = df.mean(axis=0)
    # print(df_mean)
    res_df_list.append(df)
    vals = [f"{instance}({df.shape[0]})"]
    for experiment, _ in experiment_names:
        vals.extend([round(df_mean["%s_cost"%experiment], 2),
                     round(df_mean["%s_vehicles"%experiment], 2),
                     round(df_mean[f'{experiment}_gap'], 3)])
    vals.extend([round(df_mean["sota_cost"], 2),
                 round(df_mean["sota_vehicles"], 2)])
    res_stat_list.append(vals)
df_all = pd.concat(res_df_list)
df_all.to_csv("res_all.csv", index=False)
# print(f"************ALL**********")
# print(df_all.mean(axis=0))

df_stat = pd.DataFrame(res_stat_list, columns=cols)
display(df_stat)

,instance,GES_cost,GES_vehicles,GES_gap,HYBRID_cost,HYBRID_vehicles,HYBRID_gap,HRBRID_SINGLE_cost,HRBRID_SINGLE_vehicles,HRBRID_SINGLE_gap,SOTA_cost,SOTA_vehicles
0,200(60),2651.68,13.07,-0.052,2627.07,13.03,-0.060,2635.13,13.13,-0.057,2814.63,11.57
1,400(53),6392.36,26.38,-0.042,6352.94,26.36,-0.048,6390.31,26.74,-0.042,6678.63,24.98
2,600(60),12707.14,37.07,-0.031,12543.87,36.72,-0.045,12750.49,37.85,-0.027,13128.36,34.43
3,800(60),21960.97,48.90,-0.014,21465.93,47.92,-0.043,22005.20,49.48,-0.018,22425.68,45.58
4,1000(60),33509.70,60.20,-0.008,33081.44,59.73,-0.025,34143.64,62.23,0.010,33804.99,56.95
5,ortec(212),165727.61,16.56,0.000,158504.82,16.19,-0.041,159414.26,16.30,-0.036,165727.61,16.56


In [6]:
df_mean = df.mean(axis=0)

In [11]:
df_mean

vrptw_ges_1200_vehicles                   17.024096
vrptw_ges_1200_cost                   168821.397590
vrptw_hybrid_ges_only_vehicles            16.626506
vrptw_hybrid_ges_only_cost            161535.493976
vrptw_hybrid_ges_only_240_vehicles        16.710843
vrptw_hybrid_ges_only_240_cost        161843.518072
vrptw_hybrid_ges_240_vehicles             16.638554
vrptw_hybrid_ges_240_cost             161932.638554
sota_vehicles                             17.024096
sota_cost                             168821.397590
vrptw_ges_1200_gap                         0.000000
vrptw_hybrid_ges_only_gap                 -0.040940
vrptw_hybrid_ges_only_240_gap             -0.039253
vrptw_hybrid_ges_240_gap                  -0.038651
dtype: float64